PGL monitor calibration


In [16]:
# library imports
from matplotlib import pyplot as plt
import numpy as np

# Load PGL libraries and start a PGL window
from pgl import pgl as pglLibrary
pgl = pglLibrary()

# close any existing windows
pgl.cleanUp()

# open a new window
pgl.open(0,800,600)
pgl.visualAngle(57, 60,40)

================================ pglBase: init =================================
(pgl:checkOS) Python version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:35:20) [Clang 16.0.6 ]
(pgl:checkOS) Running on MacBook Pro (MacBookPro18,3) with macOS version: 26.2
(pgl:checkOS) Apple M1 Pro Cores: 8 (6 performance and 2 efficiency) Memory: 32 GB
(pgl:checkOS) GPU: Apple M1 Pro (Built-In) 14 cores, Metal 4 support
(pgl:checkOS)   Color LCD [Main Display]: 3024 x 1964 Retina (Built-in Liquid Retina XDR Display) GammaTable size: 1024
(pglBase) Main library instance created
(pglBase:shutdownAll) Shutting down mglMetal process: 47281
(pglBase:removeOrphanedSockets) Removing orphaned socket: /Users/justin/Library/Containers/gru.mglMetal/Data/pglMetal.socket.20260131_084955.suzIeQ22KT
(pglBase:removeOrphanedSockets) Removed 1 orphaned sockets
================================= pglBase:open =================================
(pglBase:open) Starting mglMetal application: /Users/justin/Libra

Now, let's measure the temporal frequency response of the display using a photodiode. For this, you will need to do the following:

- Connect photodiode to Hamamatsu Amp via the BNC connector
- Connect the output of Hamamastu Amp to the Labjack A0 input
- Connect the USB of the Labjack to the computer
- Position the photodiode so that it is on the monitor


The following code will init the LabJack. Take a look at the output and make sure that it does not report an error loading LabJack.

In [ ]:
# import pglLabJack and create an instance
from pgl import pglLabJack
pglLabJack = pglLabJack()

We now will flicker the screen and display the output

In [ ]:
# set stimulus duration in seconds
stimulusDuration = 5

# start analog read
startTime = pgl.getSecs()
pglLabJack.startAnalogRead(duration=stimulusDuration)

# display flickering stimulus
flushTime = []
flicker = pgl.flicker(pgl, temporalFrequency=0.7, type='sin')
while pgl.getSecs() - startTime < stimulusDuration:
    flicker.display()
    flushTime.append(pgl.flush())

# stop analog read
time, data = pglLabJack.stopAnalogRead()

# plot data
pglLabJack.plotAnalogRead(time, data)

Read the gamma table and display it

In [ ]:
# Read the gamma table
gammaTable = pgl.getGammaTable(0)

# display it
plt.plot(gammaTable[0], label='Red')
plt.plot(gammaTable[1], label='Green')
plt.plot(gammaTable[2], label='Blue')
plt.title('Gamma Table')
plt.xlabel('Input Value')
plt.ylabel('Output Value')
plt.legend()
plt.show()

Set gamma table

In [18]:
# Get the size of the gamma table
displayNumber = 0
tableSize = pgl.getGammaTableSize(displayNumber)
print(f"Gamma table size: {tableSize}")

# create a gamma table
gamma = 1
lin = np.linspace(0, 1, tableSize, dtype=np.float32)
red = lin ** gamma
green = lin ** gamma
blue = lin ** gamma

# set it
pgl.setGammaTable(displayNumber, red, green, blue)

Gamma table size: 1024


Setup minolta device

- Connect ...

In [20]:
# initialize minolta calibration device
from pgl import pglCalibrationDeviceMinolta, pglCalibration
pglMinolta = pglCalibrationDeviceMinolta()

=========================== Connect Minolta CS-100A ============================
(pglCalibrationDeviceMinolta) Connect the Minolta CS-100A to the serial port
(pglCalibrationDeviceMinolta) hold down F key and turn it on.
(pglCalibrationDeviceMinolta) You should see the letter C on the Minolta display.
(pglCalibrationDeviceMinolta) Press Enter to continue...
============================== Choose Serial Port ==============================
(pglCalibrationDeviceMinolta) Select the serial port that the Minolta CS-100A is connected to.
(pglCalibrationDeviceMinolta) This should appear as something like:
(pglCalibrationDeviceMinolta)   /dev/cu.usbserial-110 - USB-Serial Controller Device
(pglSerialComm) 0: /dev/cu.debug-console - n/a
(pglSerialComm) 1: /dev/cu.JBLCharge4 - n/a
(pglSerialComm) 2: /dev/cu.Bluetooth-Incoming-Port - n/a
(pglSerialComm) 3: Cancel
(pglSerialComm) Select port [0-3]: 
(pglSerialComm) Cancelled
================================== Cancelled ===============================

Make a measurement

In [ ]:
pglMinolta.measure()

Run a calibration

In [ ]:
# initialize the calibration with the minolta device
pglCal = pglCalibration(pgl, pglMinolta)

# Run the calibration with 8 luminance steps and 2 repeats of each measurement 
pglCal.calibrate(nSteps=8, nRepeats=2)